# Solution - Project: ...
For this project, let's assume that we have measured the brain volumes for all 400 images in the OASIS dataset and stored them in a csv file. We have also access to all image data sets.

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023

Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

## Project description - Part I
For this project part we are using the data stored in a csv file.

With this data, we want to answer some crucial questions:
- ...
- ...


For this question, we can use a two-sample t-test. We will test the null hypothesis that the mean brain volumes shrinks due to Alzheimer's. The likelihood of the null hypothesis being true is assessed by calculating the t-statistic. If the magnitude of t is very far from 0, then we can reject the null hypothesis that the groups are the same.

### 1. Import Python libraries

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to ""%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
# Used to change filepaths
from pathlib import Path

# We set up matplotlib and the display function
import matplotlib.pyplot as plt
from IPython.display import display

# import numpy, pandas, pydicom and ...
import numpy as np
import pandas as pd
import pydicom

### 2. Import and read the dataset

In [ ]:
# Load OASIS csv file
df = pd.read_csv('../files/oasis_all_volumes.csv', index_col=0)

### 3. Check the dataset

In [ ]:
# Print the first five rows of the table
print(df.head())

# Print prevalence of Alzheimer's Disease
print(df.alzheimers.value_counts())

# Print a correlation table excluding non-numeric columns
print(df.select_dtypes(include='number').corr())

### 4. Testing Group Differences
Let's test the hypothesis that Alzheimer's Disease is characterized by reduced brain volume. To run the t-test, we need the brain volumes for all patients with diagnosed Alzheimer's and without in our sample. Select the 'alzheimers' values with `df.loc`, then specifying the column with "brain volume" values. For the healthy cohort, we change the selected value to 'False'. To run the t-test, import the `ttest_ind()` function from SciPy's stats module. Then, pass the two vectors as our first and second populations. The results object contains the test statistic and the p-value. The p-value corresponds to the probability that the null hypothesis is true.

In this case, the two population samples are independent from each other because they are all separate subjects.

For this exercise, use the OASIS dataset (`df`) and `ttest_ind` to evaluate the hypothesis.

In [ ]:
# Import independent two-sample t-test
from scipy.stats import ttest_ind

To better understand the function `ttest_ind` try to get more information about `ttest_ind` with the `help()` function. Which parameters does the function need? What is the output?

In [ ]:
help(ttest_ind)

Use DataFrame operations to extract brain volume data for Alzheimer's and typical groups for the column `brain_vol`.

In [ ]:
# Select data from "alzheimers" and "typical" groups
brain_alz = df.loc[df.alzheimers == True, 'brain_vol']
brain_typ = df.loc[df.alzheimers == False, 'brain_vol']

We can now perform a two-sample t-test between the brain volumes of elderly adults with and without Alzheimer's Disease. Using `results.statistic` and `results.pvalue` as your guide, answer the question: Is there strong evidence that Alzheimer's Disease is marked by smaller brain size?

In [ ]:
# Perform t-test of "alz" > "typ"
results = ttest_ind(brain_alz, brain_typ)
print('t = ', results.statistic)
print('p = ', results.pvalue)

Solution: There is some evidence for decreased brain volume in individuals with Alzheimer's Disease. Since the p-value for this t-test is greater than 0.05, we would not reject the null hypothesis that states the two groups are equal.

Visualize the distribution of brain volumes based on whether individuals have Alzheimer's disease or not.

In [ ]:
# Show boxplot of brain_vol differences
df.boxplot(column='brain_vol', by='alzheimers')
plt.show()

### 5. Normalizing metrics
We previously saw that there was not a significant difference between the brain volumes of elderly individuals with and without Alzheimer's Disease. To account for this potential confound, we can normalize brain volume with respect to skull size by calculating the brain to skull ratio.

But could a correlated measure, such as "skull volume" be masking the differences?

For this exercise, calculate a new test statistic for the comparison of brain volume between groups, after adjusting for the subject's skull size.

In [ ]:
# Adjust `brain_vol` by `skull_vol`
df['adj_brain_vol'] = df.brain_vol / df.skull_vol

Use DataFrame operations to extract brain volume data for Alzheimer's and typical groups for the new column `adj_brain_vol` and perform a two-sample t-test between the brain volumes of elderly adults with and without Alzheimer's Disease. The statistics and the p-value would be interesting here (print statistic and p value).

Using `results.statistic` and `results.pvalue` as your guide, answer the question: Is there strong evidence that Alzheimer's Disease is marked by smaller brain size, relative to skull size?

In [ ]:
# Select brain measures by group
brain_alz = df.loc[df.alzheimers == True, 'adj_brain_vol']
brain_typ = df.loc[df.alzheimers == False, 'adj_brain_vol']

# Evaluate null hypothesis
results = ttest_ind(brain_alz, brain_typ)
print('t = ', results.statistic)
print('p = ', results.pvalue)

Solution: Yes, reject the null hypothesis! Based on the results.statistic and results.pvalue.

## Project description - Part II
For this project part ...